Автор: [Виктория Фирсанова](https://github.com/vifirsanova)

[Репозиторий](https://github.com/vifirsanova/TM)

# Импорт библиотек

In [ ]:
# цветной вывод
!pip install colorama

In [ ]:
# последняя версия spacy
!pip install -U spacy

In [ ]:
# spacy для русского языка
!spacy download ru_core_news_sm

In [ ]:
import pandas as pd
import json
from colorama import Fore, Style

import random

import re

from collections import defaultdict

import numpy as np

import nltk
from nltk.corpus import stopwords

import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel

from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer

#import spacy
#import ru_core_news_sm

from pprint import pprint

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


# Загрузка данных

Для обучения используем собственный [корпус новостей газеты Московский комсомолец за 2020-2021 года.](https://drive.google.com/file/d/1ebxe77hTEWvmUggCMnRGdrVDwhuxC9Mr/view)

In [ ]:
# подгружаем корпус из GitHub
!wget https://raw.githubusercontent.com/vifirsanova/W2V/main/corpus.json

--2021-12-19 16:49:02--  https://raw.githubusercontent.com/vifirsanova/W2V/main/corpus.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 92111843 (88M) [text/plain]
Saving to: ‘corpus.json.1’

corpus.json.1       100%[===================>]  87.84M   149MB/s    in 0.6s    

2021-12-19 16:49:10 (149 MB/s) - ‘corpus.json.1’ saved [92111843/92111843]



In [ ]:
# преобразование данных корпуса в таблицу csv
with open('corpus.json') as json_file:
    data = json.load(json_file)
df = pd.read_json('corpus.json')
print(f"{Fore.RED}Список тем в корпусе:{Style.RESET_ALL} {df.rubric.unique()}")
df.head()

Список тем в корпусе: ['Общество' 'Происшествия' 'Политика' 'Москва' 'Экономика' 'Наука'
 'В мире' 'Спорт' 'Московская область' 'Культура' 'Армия' 'Авто' 'Космос'
 'Светская жизнь' 'Технологии' 'Рунет' 'Кошелек' 'Футбол' 'Катастрофы'
 'Образование' 'Письма президенту' 'Здоровье' 'Московская премьера'
 'Свежий МК' 'Турклуб' 'Криминал']


,link,title,date,time,rubric,text
0,https://www.mk.ru/social/2020/03/31/minobrnauk...,Минобрнауки решило перенести вступительные экз...,31.03.2020,09:11,Общество,\nВ Минобрнауки РФ приняли решение перенести д...
1,https://www.mk.ru/social/2020/08/17/razveyan-m...,Развеян миф о риске заразиться COVID-19 через ...,17.08.2020,09:33,Общество,\nВсемирная организация здравоохранения (ВОЗ) ...
2,https://www.mk.ru/incident/2020/05/05/v-gaage-...,В Гааге задержали десятки протестующих против ...,05.05.2020,17:12,Происшествия,\nПолиция Нидерландов пресекла акцию протеста ...
3,https://www.mk.ru/social/2021/03/23/v-kremle-p...,В Кремле прокомментировали возможную отставку ...,23.03.2021,13:05,Общество,\nПресс-секретарь российского президента Дмитр...
4,https://www.mk.ru/social/2021/03/01/moskva-i-p...,Москва и Петербург не попали в топ-50 городов ...,01.03.2021,12:17,Общество,\nСоставлен рейтинг 100 лучших городов мира по...


In [ ]:
# сохраняем корпус в переменную
with open('corpus.json', encoding='utf-8', newline='') as json_file:
    data = json.load(json_file)

In [ ]:
# собираем тексты из корпуса
full_dataset = []

for elem in data['data']:
    full_dataset.append(elem['text'])

# случайное перемешивание данных
random.shuffle(full_dataset)

# деление на обучающую (80% данных) и проверочную (20% данных) выборку
TRAIN_VAL_SPLIT = int(len(full_dataset) * 0.8)
train = full_dataset[:TRAIN_VAL_SPLIT]
val = full_dataset[TRAIN_VAL_SPLIT:]

# Токенизация

In [18]:
# удаляем латинские буквы, т.к. корпус содержит русскоязычные новости, а вставки на латинице могут содержать ссылки, элементы сайта (например, рекламные вставки) и другие побочные элементы
val_text = [re.sub('[a-zA-Z]', '', sent) for sent in val]
train_text = [re.sub('[a-zA-Z]', '', sent) for sent in train]
# создаем функцию для токенизации
def tokenize(data):
    for text in data:
        # для токенизации используем simple_preprocess из gensim
        yield(gensim.utils.simple_preprocess(str(text).encode('utf-8'), deacc=True))  # deacc удаляет знаки препинания

val_tokenized = list(tokenize(val_text))
train_tokenized = list(tokenize(train))
print(f"{Fore.RED}Образец токенизированного текста{Style.RESET_ALL}\n", val_tokenized[0])

Образец токенизированного текста
 ['количество', 'зараженных', 'коронавирусом', 'нового', 'типа', 'на', 'борту', 'карантинного', 'судна', 'побережья', 'японии', 'увеличилось', 'на', 'человек', 'составив', 'сообщает', 'телеканал', 'ранее', 'сообщалось', 'что', 'общеи', 'сложности', 'заболели', 'пассажиров', 'членов', 'экипажа', 'из', 'около', 'человек', 'на', 'борту', 'круизного', 'судна', 'первоначально', 'карантин', 'планировался', 'до', 'февраля', 'однако', 'может', 'быть', 'продлен', 'стало', 'известно', 'что', 'на', 'борту', 'лаинер', 'находятся', 'гражданина', 'россии', 'заражении', 'их', 'вирусом', 'информации', 'не', 'поступало', 'читаите', 'также', 'запертыи', 'на', 'лаинере', 'коронавирусом', 'россиянин', 'описал', 'обстановку']


# Стоп-слова

In [19]:
# загрузка стоп-слов из репозитория
!wget https://raw.githubusercontent.com/vifirsanova/W2V/main/swl.txt

# стоп-слова 
stop_words = []

with open('swl.txt', encoding='utf-8') as f:
    for word in f:
      stop_words.append(word[:-1])

print(f"{Fore.RED}Образец стоп-слов:{Style.RESET_ALL}", stop_words[6:13])

def build_vocab(data_tokenized, pad_word=None, stop_words=stop_words):
    freq_dict = defaultdict(int) # количество словоупотреблений в документах 
    doc_n = len(data_tokenized) # общее количество документов

    # посчитать количество словоупотреблений в документах 
    for text in data_tokenized:
        unique_text_tokens = set(text)
        for token in unique_text_tokens:
            freq_dict[token] += 1


    # удалить стоп-слова
    for word in list(freq_dict.keys()):
        if word in stop_words:
            freq_dict.pop(word)


    # удалить уникальные слова
    freq_dict = {word: count for word, count in freq_dict.items() if count > 1}

    # отсортировать слова по убыванию частоты
    sorted_freq_dict = sorted(freq_dict.items(),
                                reverse=True,
                                key=lambda x: x[1])

    # паддинг
    padded_freq_dict = [(pad_word, 0)] + sorted_freq_dict
    
    # обновить нумерацию слов
    word2id = {word: i for i, (word, _) in enumerate(padded_freq_dict)}

    # преобразуем частотный словарь к виду матриц чисел
    word2freq = np.array([count / doc_n for _, count in padded_freq_dict], dtype='float32')

    return word2id, word2freq


# строим словарь для обучающей выборки
vocabulary, word_doc_freq = build_vocab(train_tokenized, pad_word='<PAD>', stop_words=stop_words)

print(f"{Fore.RED}Размер словаря:{Style.RESET_ALL} {len(vocabulary)}")
print(f"{Fore.RED}Образец словаря{Style.RESET_ALL}\n", list(vocabulary.items())[:10])

--2021-12-19 16:56:57--  https://raw.githubusercontent.com/vifirsanova/W2V/main/swl.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10814 (11K) [text/plain]
Saving to: ‘swl.txt.5’

swl.txt.5           100%[===================>]  10.56K  --.-KB/s    in 0s      

2021-12-19 16:56:57 (46.6 MB/s) - ‘swl.txt.5’ saved [10814/10814]

Образец стоп-слов: ['абы', 'авось', 'ага', 'аж', 'аз', 'ай', 'ай-ай-ай']
Размер словаря: 35345
Образец словаря
 [('<PAD>', 0), ('этом', 1), ('россии', 2), ('ранее', 3), ('заявил', 4), ('словам', 5), ('будет', 6), ('время', 7), ('коронавируса', 8), ('сообщает', 9)]


# Биграммная модель / триграммная модель

In [ ]:
# Используем Phrases из gensim
# на вход модель принимает токенизированные данные
# чем выше min_count и threshold, тем меньше би- или триграм будет устанавливать модель
bigram = gensim.models.Phrases(train_tokenized, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[train_tokenized], threshold=100)  

bi_model = gensim.models.phrases.Phraser(bigram)
tri_model = gensim.models.phrases.Phraser(trigram)

print(f"{Fore.RED}Образец работы биграммной модели{Style.RESET_ALL}\n", bi_model[bi_model[train_tokenized[0]]])
print(f"{Fore.RED}Образец работы триграммной модели{Style.RESET_ALL}\n", tri_model[tri_model[train_tokenized[0]]])

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Образец работы биграммной модели
 ['глава', 'крыма', 'сергеи_аксенов', 'заявил', 'что', 'декабря', 'будет', 'регионе', 'выходным_днем', 'для', 'госслужащих', 'сотрудников', 'всех', 'организации', 'учреждении', 'ранее', 'четверг', 'аналогичное', 'решение', 'было_принято', 'курскои', 'владимирскои', 'областях', 'днем', 'ранее', 'белгородскои', 'области', 'читаите', 'также', 'чиновник', 'выругался', 'матом', 'на', 'совещании', 'главои', 'крыма']
Образец работы триграммной модели
 ['глава', 'крыма_сергеи_аксенов', 'заявил', 'что', 'декабря', 'будет', 'регионе', 'выходным', 'днем', 'для', 'госслужащих', 'сотрудников', 'всех', 'организации', 'учреждении', 'ранее', 'четверг', 'аналогичное', 'решение_было_принято', 'курскои', 'владимирскои', 'областях', 'днем', 'ранее', 'белгородскои_области', 'читаите_также', 'чиновник', 'выругался', 'матом', 'на', 'совещании', 'главои', 'крыма']


# Преобработка

In [ ]:
def remove_stopwords(data):
    """
    Удаление стоп-слов
    :data: токенизированные тексты (list)
    :return: токенизированные тексты, очищенные от стоп-слов (list)
    """
    return [[word for word in text if word not in stop_words] for text in data]


def bigrams(data):
    """
    Построение биграмм
    :data: токенизированные тексты (list)
    :return: результат работы биграммной модели (list)
    """
    return [bi_model[text] for text in data]

def trigrams(data):
    """
    Построение биграмм
    :data: токенизированные тексты (list)
    :return: результат работы биграммной модели (list)
    """
    return [tri_model[text] for text in data]


# Лемматизация с помощью Spacey
# здесь мы ее пропустили из-за объема корпуса
##def lemmatization(data):
##    nlp = spacy.load('ru_core_news_sm')
##    data_lemmatized = []
##    for sent in data:
##        text = nlp(" ".join(sent)) 
##        data_lemmatized.append([token.lemma_ for token in text])
##    return data_lemmatized


data_clean = remove_stopwords(train_tokenized)
data_bigrams = bigrams(data_clean)
data_trigrams = trigrams(data_clean)
##data_lemmatized = lemmatization(data_bigrams)


print(f"{Fore.RED}Образец очищенных от стоп-слов данных:{Style.RESET_ALL}\n", data_clean[0])
print(f"{Fore.RED}Образец обработанных данных c биграммами:{Style.RESET_ALL}\n", data_bigrams[0])
print(f"{Fore.RED}Образец обработанных данных c биграммами:{Style.RESET_ALL}\n", data_trigrams[0])
##print(f"{Fore.RED}Образец лемматизированных данных:{Style.RESET_ALL}\n", data_lemmatized[0])

Образец очищенных от стоп-слов данных:
 ['глава', 'крыма', 'сергеи', 'аксенов', 'заявил', 'декабря', 'будет', 'регионе', 'выходным', 'днем', 'госслужащих', 'сотрудников', 'всех', 'организации', 'учреждении', 'ранее', 'четверг', 'аналогичное', 'решение', 'принято', 'курскои', 'владимирскои', 'областях', 'днем', 'ранее', 'белгородскои', 'области', 'читаите', 'чиновник', 'выругался', 'матом', 'совещании', 'главои', 'крыма']
Образец обработанных данных c биграммами:
 ['глава', 'крыма', 'сергеи_аксенов', 'заявил', 'декабря', 'будет', 'регионе', 'выходным_днем', 'госслужащих', 'сотрудников', 'всех', 'организации', 'учреждении', 'ранее', 'четверг', 'аналогичное', 'решение_принято', 'курскои', 'владимирскои', 'областях', 'днем', 'ранее', 'белгородскои', 'области', 'читаите', 'чиновник', 'выругался', 'матом', 'совещании', 'главои', 'крыма']
Образец обработанных данных c биграммами:
 ['глава', 'крыма', 'сергеи_аксенов', 'заявил', 'декабря', 'будет', 'регионе', 'выходным', 'днем', 'госслужащих', 

# Обучающий словарь и Term Frequency

In [32]:
# создание биграммного словаря с помощью gensim.corpora
id2word = corpora.Dictionary(data_bigrams)

# TF
corpus = [id2word.doc2bow(text) for text in data_bigrams]

print(f"{Fore.RED}Образец элементов словаря:{Style.RESET_ALL}\n", id2word[0], id2word[1], id2word[2])
print(f"{Fore.RED}Образец обучающего корпуса:{Style.RESET_ALL}\n", corpus[0])
print(f"{Fore.RED}Term Frequency:{Style.RESET_ALL}\n", [[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]])

Образец элементов словаря:
 борту борту_круизного вирусом
Образец обучающего корпуса:
 [(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 2), (33, 1), (34, 1), (35, 2), (36, 1), (37, 1), (38, 1)]
Term Frequency:
 [[('борту', 2), ('борту_круизного', 1), ('вирусом', 1), ('гражданина', 1), ('заболели', 1), ('запертыи', 1), ('заражении', 1), ('зараженных_коронавирусом', 1), ('информации', 1), ('карантин', 1), ('карантинного', 1), ('количество', 1), ('коронавирусом', 1), ('лаинер', 1), ('лаинере', 1), ('находятся', 1), ('нового', 1), ('обстановку', 1), ('общеи_сложности', 1), ('описал', 1), ('пассажиров', 1), ('первоначально', 1), ('планировался', 1), ('побережья', 1), ('поступало', 1), ('продлен', 1), ('ранее_сообщалось', 1), ('россии', 1), ('россиянин', 1), ('

In [34]:
# создание триграммного словаря с помощью gensim.corpora
id2word = corpora.Dictionary(data_trigrams)

# TF
tri_corpus = [id2word.doc2bow(text) for text in data_trigrams]

print(f"{Fore.RED}Образец элементов словаря:{Style.RESET_ALL}\n", id2word[0], id2word[1], id2word[2])
print(f"{Fore.RED}Образец обучающего корпуса:{Style.RESET_ALL}\n", tri_corpus[0])
print(f"{Fore.RED}Term Frequency:{Style.RESET_ALL}\n", [[(id2word[id], freq) for id, freq in cp] for cp in tri_corpus[:1]])

Образец элементов словаря:
 борту борту_круизного вирусом
Образец обучающего корпуса:
 [(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 2), (33, 1), (34, 1), (35, 2), (36, 1), (37, 1), (38, 1)]
Term Frequency:
 [[('борту', 2), ('борту_круизного', 1), ('вирусом', 1), ('гражданина', 1), ('заболели', 1), ('запертыи', 1), ('заражении', 1), ('зараженных_коронавирусом', 1), ('информации', 1), ('карантин', 1), ('карантинного', 1), ('количество', 1), ('коронавирусом', 1), ('лаинер', 1), ('лаинере', 1), ('находятся', 1), ('нового', 1), ('обстановку', 1), ('общеи_сложности', 1), ('описал', 1), ('пассажиров', 1), ('первоначально', 1), ('планировался', 1), ('побережья', 1), ('поступало', 1), ('продлен', 1), ('ранее_сообщалось', 1), ('россии', 1), ('россиянин', 1), ('

# Модель LDA

In [ ]:
# обучение модели LDA с gensim, биграммы
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# обучение модели LDA с gensim, триграммы
tri_lda_model = gensim.models.ldamodel.LdaModel(corpus=tri_corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# результат, биграммы
pprint(lda_model.print_topics())

[(0,
  '0.010*"исключить" + 0.007*"вооружения" + 0.006*"инфицированным" + '
  '0.005*"стриминговых" + 0.005*"обжаловании" + 0.005*"будет" + '
  '0.005*"областях" + 0.004*"котором" + 0.004*"конфликту" + '
  '0.004*"исламскои_республики"'),
 (1,
  '0.008*"выкладывали" + 0.007*"организовали" + 0.006*"него" + '
  '0.006*"увеличат" + 0.006*"им" + 0.005*"лет" + 0.005*"дал" + 0.005*"данным" '
  '+ 0.005*"лице" + 0.004*"святыня"'),
 (2,
  '0.011*"смотрите_видео" + 0.007*"продолжается" + 0.006*"хюэ" + 0.005*"проб" '
  '+ 0.005*"специально" + 0.005*"премьер_министр" + 0.005*"россииских" + '
  '0.005*"ступин" + 0.004*"навального" + 0.004*"вируса"'),
 (3,
  '0.006*"заражения_коронавирусом" + 0.005*"сообщает_рен" + '
  '0.005*"производство" + 0.005*"министр_обороны" + 0.004*"актеру" + '
  '0.004*"указаны" + 0.004*"ступин" + 0.004*"ужасающеи" + 0.003*"тяжело" + '
  '0.003*"человечеству"'),
 (4,
  '0.019*"трампа" + 0.015*"заявил" + 0.013*"прибыл" + 0.012*"принять" + '
  '0.009*"регионе" + 0.009*"тран

In [ ]:
# результат, триграммы
pprint(tri_lda_model.print_topics())

[(0,
  '0.006*"александр" + 0.005*"считает" + 0.005*"есть" + 0.005*"партии" + '
  '0.004*"нужно" + 0.004*"фото" + 0.004*"делать" + 0.004*"ес" + '
  '0.004*"актриса" + 0.003*"можно"'),
 (1,
  '0.026*"сша" + 0.019*"президент" + 0.012*"президента" + 0.010*"украины" + '
  '0.009*"заявил" + 0.006*"владимир_путин" + 0.005*"россия" + 0.004*"стороны" '
  '+ 0.004*"путин" + 0.004*"назвал"'),
 (2,
  '0.010*"сообщили" + 0.009*"сообщает" + 0.006*"результате" + 0.006*"были" + '
  '0.005*"мужчина" + 0.005*"отмечается" + 0.004*"города" + 0.004*"раионе" + '
  '0.004*"полиции" + 0.004*"произошло"'),
 (3,
  '0.011*"года" + 0.010*"белоруссии" + 0.009*"лукашенко" + 0.008*"стране" + '
  '0.005*"мире" + 0.005*"долларов" + 0.004*"тысяч" + 0.004*"процентов" + '
  '0.004*"данным" + 0.004*"заболевших"'),
 (4,
  '0.008*"была" + 0.006*"время" + 0.005*"стала" + 0.005*"видео" + 0.004*"теме" '
  '+ 0.004*"вместе" + 0.004*"которои" + 0.004*"пишет" + 0.004*"женщина" + '
  '0.003*"акции"'),
 (5,
  '0.024*"человек" + 0.

In [ ]:
# U-Mass, биграммы
coherence_model_lda = CoherenceModel(model=lda_model, corpus=corpus, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('U-Mass: ', coherence_lda)

U-Mass:  -4.730274166899538


In [ ]:
# U-Mass, триграммы
tri_coherence_model_lda = CoherenceModel(model=tri_lda_model, corpus=tri_corpus, coherence='u_mass')
tri_coherence_lda = tri_coherence_model_lda.get_coherence()
print('U-Mass: ', tri_coherence_lda)

U-Mass:  -4.191770845441544


In [ ]:
# Перплексия, биграммы
perplexity = lda_model.log_perplexity(corpus)
print('Перплексия: ', perplexity)

# Когерентность, биграммы
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_bigrams, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Когерентность: ', coherence_lda)

Перплексия:  -10.842545018482339
Когерентность:  0.3045778313558452


In [ ]:
# Перплексия, триграммы
tri_perplexity = tri_lda_model.log_perplexity(corpus)
print('Перплексия: ', tri_perplexity)

# Когерентность, триграммы
tri_coherence_model_lda = CoherenceModel(model=tri_lda_model, texts=data_trigrams, dictionary=id2word, coherence='c_v')
tri_coherence_lda = tri_coherence_model_lda.get_coherence()
print('Когерентность: ', tri_coherence_lda)

Перплексия:  -14.366108945925761
Когерентность:  0.3473169520826572


# Модель LSA

In [ ]:
# обучение модели LSA с gensim, биграммы
lsa_model = gensim.models.lsimodel.LsiModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                           decay=1.0,
                                           distributed=False,
                                           chunksize=100,
                                           power_iters=2,
                                           extra_samples=100)

In [ ]:
# обучение модели LSA с gensim, триграммы
tri_lsa_model = gensim.models.lsimodel.LsiModel(corpus=tri_corpus,
                                           id2word=id2word,
                                           num_topics=5,
                                           decay=1.0,
                                           distributed=False,
                                           chunksize=100,
                                           power_iters=2,
                                           extra_samples=100)

In [ ]:
# результат, биграммы
pprint(lsa_model.print_topics())

[(0,
  '0.338*"санкт_петербург" + 0.323*"был" + 0.207*"сергеи_шоигу" + '
  '0.202*"будет" + 0.194*"заявил" + 0.192*"трампа" + 0.174*"регионе" + '
  '0.161*"подчеркнул" + 0.139*"государственным" + 0.138*"ударит"'),
 (1,
  '-0.589*"санкт_петербург" + -0.301*"трампа" + 0.198*"был" + 0.170*"хюэ" + '
  '0.157*"будет" + -0.136*"сведения" + -0.125*"прибыл" + 0.121*"ударит" + '
  '-0.113*"заявил" + 0.109*"данным"'),
 (2,
  '0.427*"санкт_петербург" + -0.366*"трампа" + 0.319*"государственным" + '
  '0.291*"юге" + -0.217*"заявил" + -0.216*"сергеи_шоигу" + 0.159*"сведения" + '
  '-0.157*"подчеркнул" + 0.141*"будет" + -0.127*"принять"'),
 (3,
  '-0.579*"будет" + 0.319*"юге" + 0.300*"государственным" + 0.271*"трампа" + '
  '-0.161*"взять" + 0.130*"владимир_путин" + 0.126*"выкладывали" + '
  '-0.118*"подчеркнул" + -0.088*"сергеи_шоигу" + 0.081*"увеличат"'),
 (4,
  '0.479*"трампа" + 0.452*"будет" + -0.324*"сергеи_шоигу" + '
  '-0.236*"санкт_петербург" + 0.199*"юге" + 0.158*"государственным" + '
  '-0.

In [ ]:
# результат, триграммы
pprint(tri_lsa_model.print_topics())

[(0,
  '0.334*"россии" + 0.322*"этом" + 0.209*"словам" + 0.205*"будет" + '
  '0.189*"сша" + 0.189*"заявил" + 0.170*"ранее" + 0.163*"отметил" + '
  '0.144*"года" + 0.142*"уже"'),
 (1,
  '0.521*"россии" + -0.288*"будет" + 0.227*"сша" + 0.187*"рф" + '
  '-0.142*"кислорода" + -0.119*"уже" + -0.117*"этом" + -0.109*"можно" + '
  '-0.098*"словам" + -0.097*"время"'),
 (2,
  '0.323*"году" + -0.272*"словам" + 0.260*"россии" + 0.222*"года" + '
  '0.180*"кислорода" + -0.149*"сша" + 0.141*"ани" + -0.137*"этом" + '
  '0.136*"будет" + 0.129*"развития"'),
 (3,
  '0.365*"кислорода" + -0.223*"году" + -0.196*"ани" + 0.168*"кислород" + '
  '0.141*"технологии" + 0.140*"марсе" + 0.139*"дыхания" + 0.131*"сша" + '
  '0.112*"марса" + 0.112*"астронавтов"'),
 (4,
  '-0.272*"развития" + 0.264*"ани" + 0.258*"году" + -0.179*"территории" + '
  '-0.147*"парк" + -0.145*"парка" + -0.144*"конкурса" + 0.139*"город" + '
  '-0.127*"города" + 0.122*"нашеи_эры"')]


In [ ]:
# U-Mass, биграммы
coherence_model_lsa = CoherenceModel(model=lsa_model, corpus=corpus, coherence='u_mass')
coherence_lsa = coherence_model_lsa.get_coherence()
print('U-Mass: ', coherence_lsa)

U-Mass:  -4.530492492598545


In [ ]:
# U-Mass, триграммы
tri_coherence_model_lsa = CoherenceModel(model=tri_lsa_model, corpus=tri_corpus, coherence='u_mass')
tri_coherence_lsa = tri_coherence_model_lsa.get_coherence()
print('U-Mass: ', tri_coherence_lsa)

U-Mass:  -4.910205035773785


In [ ]:
# Когерентность, биграммы
coherence_model_lda = CoherenceModel(model=lsa_model, texts=data_bigrams, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Когерентность: ', coherence_lsa)

Когерентность:  -4.530492492598545


In [ ]:
# Когерентность, триграммы
tri_coherence_model_lsa = CoherenceModel(model=tri_lsa_model, texts=data_trigrams, dictionary=id2word, coherence='c_v')
tri_coherence_lsa = tri_coherence_model_lda.get_coherence()
print('Когерентность: ', tri_coherence_lsa)

Когерентность:  0.3473169520826572


# Модель NMF

In [ ]:
# обучение модели NMF с sklearn, биграммы
vect = TfidfVectorizer(min_df=1, stop_words=None)
X = vect.fit_transform([str(data_bigrams)])

nmf_model = NMF(n_components=10, random_state=5)
nmf_model.fit(X)
nmf_features = nmf_model.transform(X)

components_df = pd.DataFrame(nmf_model.components_, columns=vect.get_feature_names())

In [ ]:
# обучение модели NMF с sklearn, биграммы
tri_vect = TfidfVectorizer(min_df=1, stop_words=None)
tri_X = tri_vect.fit_transform([str(data_trigrams)])

tri_nmf_model = NMF(n_components=10, random_state=5)
tri_nmf_model.fit(tri_X)
tri_nmf_features = tri_nmf_model.transform(tri_X)

tri_components_df = pd.DataFrame(tri_nmf_model.components_, columns=tri_vect.get_feature_names())

In [ ]:
# результаты, биграммы
for topic in range(components_df.shape[0]):
    tmp = components_df.iloc[topic]
    print(f'For topic {topic+1} the words with the highest value are:')
    print(tmp.nlargest(10))
    print('\n')

For topic 1 the words with the highest value are:
этом       9.840106
россии     9.195706
заявил     6.167439
ранее      5.993992
будет      5.866784
словам     5.718336
сша        5.435128
человек    5.247714
года       4.604223
уже        4.409506
Name: 0, dtype: float64


For topic 2 the words with the highest value are:
последствия              0.036013
объемы                   0.035271
безопасность             0.034147
градусов_тепла           0.033273
исхода                   0.031907
король                   0.030472
министерство_финансов    0.030022
борьбу                   0.029782
нагрузки                 0.029703
источников               0.029055
Name: 1, dtype: float64


For topic 3 the words with the highest value are:
харьковскои         0.033070
политическое        0.033038
предлагает          0.032228
владимиру_путину    0.032181
прежнии             0.032115
аякс                0.031519
биатлониста         0.030777
авиация             0.030744
торговые            0.0305

In [ ]:
# результаты, триграммы
for topic in range(tri_components_df.shape[0]):
    tmp = tri_components_df.iloc[topic]
    print(f'For topic {topic+1} the words with the highest value are:')
    print(tmp.nlargest(10))
    print('\n')

For topic 1 the words with the highest value are:
этом       11.496643
россии     11.278685
заявил      7.566694
ранее       7.355240
будет       7.157317
словам      7.013747
сша         6.693593
человек     6.263553
года        5.563700
уже         5.417258
Name: 0, dtype: float64


For topic 2 the words with the highest value are:
ставропольского_края    0.036838
объявили                0.035579
главному                0.033970
градоначальник          0.033564
министр_обороны         0.030284
спортивные              0.030145
предметов               0.029719
морозова                0.029569
поддержка               0.029396
источник_экстренных     0.029309
Name: 1, dtype: float64


For topic 3 the words with the highest value are:
таком_случае       0.034678
анализа            0.033336
политолог          0.033327
торговом           0.030798
ходу               0.030372
подобных           0.029571
расставания        0.028957
утечка             0.028873
распространение    0.028525
имел  

# Вектора темообразующих слов

In [ ]:
word2vec.wv.most_similar('астронавты')

[('бенкен', 0.8296997547149658),
 ('херли', 0.8059341907501221),
 ('космонавты', 0.7791233658790588),
 ('рубинс', 0.7277292013168335),
 ('астронавт', 0.7082988023757935),
 ('кэссиди', 0.702535092830658),
 ('crew', 0.6957943439483643),
 ('астронавтами', 0.6944085359573364),
 ('скрипочка', 0.6858607530593872),
 ('dragon', 0.6788076162338257)]

In [ ]:
word2vec.wv.most_similar('белоруссия')

[('турция', 0.7059727907180786),
 ('молдавия', 0.705755352973938),
 ('польша', 0.7039617300033569),
 ('киргизия', 0.6994829773902893),
 ('румыния', 0.6741063594818115),
 ('россия', 0.6676888465881348),
 ('сирия', 0.6670364141464233),
 ('венгрия', 0.6631976366043091),
 ('куба', 0.661466121673584),
 ('литва', 0.6543489694595337)]

In [ ]:
word2vec.wv.most_similar('вирус')

[('грипп', 0.7529324293136597),
 ('мутировать', 0.7492635250091553),
 ('мутирует', 0.7478526830673218),
 ('мутировавший', 0.7448064088821411),
 ('штамм', 0.7445308566093445),
 ('циркулировать', 0.7365777492523193),
 ('возбудитель', 0.7199515104293823),
 ('инфекция', 0.7106943130493164),
 ('вируса', 0.7069671750068665),
 ('передался', 0.7054847478866577)]